# 誤差逆伝搬法

重みパラメータの勾配の計算を数値微分でやると、計算時間がかかる問題がある。
勾配の計算を効率良く行う手法に誤差逆伝搬法がある。

誤差逆伝搬法を理解するのに数式を用いるのと、計算グラフ(computational graph)を用いる方法があるが、後者を用いる。

## 計算グラフ

計算の過程をグラフで表す。グラフはデータ構造としてのグラフで、複数のノードとエッジからなる。

計算グラフは左から右へ順伝搬で計算される。その逆の逆伝搬は微分を計算するにあたり重要になる。

計算グラフは局所的な計算を伝搬していく。例えば加算ノードは2つの数の和を出力するが、その数がどのように計算されたかは関係ない。

局所的な計算と、途中の計算結果をすべて保持できる利点がある。

## 連鎖率

### 連鎖率とは

合成関数は複数の関数によって構成される関数のこと。たとえば、$z = (x + y)^2$は次の2つの式で構成される。

$$
z = t^2 \\
t = x + y
$$

連鎖率とは合成関数の微分についての性質で、次のように定義される。

> ある関数が合成関数で表される場合、その合成関数の微分は、合成関数を構成するそれぞれの関数の微分の積によって表すことができる

先程の数式について適用すると、 $\partial t$が打ち消し合うことから簡単に理解できる。

$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x}
$$

$$
\frac{\partial z}{\partial x} = \frac{\partial z}{\partial t} \frac{\partial t}{\partial x} = 2t \cdot 1 = 2 (x + y)
$$

## 逆伝搬

加算ノードの逆伝搬は次のノードに入力信号をそのまま出力するだけ。

乗算ノードは、逆伝搬の際入力信号をひっくり返して伝搬する。そのため、乗算ノードを実装する際は順伝搬の入力信号を保持する。

## 単純なレイヤの実装

例で挙げたりんごの買い物を実装する。乗算ノードを乗算レイヤ（MulLayer）、加算ノードを加算レイヤ（AddLayer)という名前にする。

In [2]:
class MulLayer:
    def __init__(self):
        self.x = None # null のようなもの、 a is None, a is not Noneのように比較する。==は使わない
        self.y = None

    def forward(self, x, y):
        self.x = x
        self.y = y                
        out = x * y

        return out

    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x

        return dx, dy


class AddLayer:
    def __init__(self):
        pass # 何も行わないという意味

    def forward(self, x, y):
        out = x + y

        return out

    def backward(self, dout):
        dx = dout * 1
        dy = dout * 1

        return dx, dy

In [3]:
apple = 100
apple_num = 2
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

price

220.00000000000003

In [4]:
# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)
dapple, dapple_num, dtax

(2.2, 110.00000000000001, 200)

In [5]:
from layer_naive import *

apple = 100
apple_num = 2
orange = 150
orange_num = 3
tax = 1.1

# layer
mul_apple_layer = MulLayer()
mul_orange_layer = MulLayer()
add_apple_orange_layer = AddLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)  # (1)
orange_price = mul_orange_layer.forward(orange, orange_num)  # (2)
all_price = add_apple_orange_layer.forward(apple_price, orange_price)  # (3)
price = mul_tax_layer.forward(all_price, tax)  # (4)

# backward
dprice = 1
dall_price, dtax = mul_tax_layer.backward(dprice)  # (4)
dapple_price, dorange_price = add_apple_orange_layer.backward(dall_price)  # (3)
dorange, dorange_num = mul_orange_layer.backward(dorange_price)  # (2)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)  # (1)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dOrange:", dorange)
print("dOrange_num:", int(dorange_num))
print("dTax:", dtax)

price: 715
dApple: 2.2
dApple_num: 110
dOrange: 3.3000000000000003
dOrange_num: 165
dTax: 650
